In [408]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# load secrets
load_dotenv()
import math
import json
from IPython.display import JSON


In [409]:
# iex cloud - api data request
iex_key = os.getenv("IEX_API_KEY")
url = 'https://cloud.iexapis.com/stable/ref-data/symbols?token=' + iex_key
response = requests.get(url)
data = response.json()
symbols_df = pd.DataFrame(data)
# print 
symbols_df.head()

,symbol,exchange,exchangeSuffix,exchangeName,exchangeSegment,exchangeSegmentName,name,date,type,iexId,region,currency,isEnabled,figi,cik,lei
0,A,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Agilent Technologies Inc.,2024-02-20,cs,IEX_46574843354B2D52,US,USD,True,BBG000C2V3D6,0001090872,QUIX8Y7A2WP0XRMW7G29
1,AA,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Alcoa Corp,2024-02-20,cs,IEX_4238333734532D52,US,USD,True,BBG00B3T3HD3,0001675149,549300T12EZ1F6PWWU29
2,AAA,ARCX,,Nyse Arca,ARCX,Nyse Arca,Investment Managers Series Trust II - AXS Firs...,2024-02-20,et,IEX_5030314338392D52,US,USD,True,BBG01B0JRCS6,0001587982,549300SU7ER9OFETRU41
3,AAAU,BATS,,Cboe Bzx U S Equities Exchange,BATS,Cboe Bzx U S Equities Exchange,Goldman Sachs Physical Gold ETF Trust - Goldma...,2024-02-20,et,IEX_474B433136332D52,US,USD,True,BBG00LPXX872,0001708646,None
4,AACG,XNAS,,Nasdaq All Markets,XNMS,Nasdaq Nms Global Market,ATA Creativity Global - ADR,2024-02-20,ad,IEX_44595A4C53392D52,US,USD,True,BBG000V2S3P6,0001420529,None


In [410]:
# NYSE and NASDAQ symbols only (filter)
filtered_df = symbols_df[symbols_df['exchange'].isin(['XNYS', 'XNAS'])]

# select cols
filtered_df = filtered_df[['symbol', 'exchange', 'exchangeName']]

# filter df
print(filtered_df.head())
data_size = len(filtered_df.index)
print()
print(f"Data size: {data_size}") # get length of dataframe

  symbol exchange                 exchangeName
0      A     XNYS  New York Stock Exchange Inc
1     AA     XNYS  New York Stock Exchange Inc
4   AACG     XNAS           Nasdaq All Markets
5   AACI     XNAS           Nasdaq All Markets
6  AACIU     XNAS           Nasdaq All Markets

Data size: 7791


In [411]:
batch_size = 100
# Assuming filtered_df is a DataFrame that contains a 'symbol' column.
# Calculate total batches
total_batches = math.ceil(len(filtered_df) / batch_size)
print(f"Total batches: {total_batches}")

combined_data = []

# function -> fetch market cap data in batches
def fetch_market_cap(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# Initialize a counter for completed batches
completed_batches = 0

# Define the length of the progress bar
progress_bar_length = 50

# batch processing - IMPORTANT (API efficiency)
for i in range(0, len(filtered_df), batch_size):
    batch_symbols = filtered_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_market_cap(batch_symbols, iex_key)  # Make sure iex_key is defined and valid
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        market_cap = batch_data.get(symbol, {}).get('quote', {}).get('marketCap', None)
        combined_data.append({'symbol': symbol, 'marketcap': market_cap})
    
    # Increment the completed batches counter
    completed_batches += 1

    # Calculate the progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # Create the progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # Print the progress bar with the percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Ensure the next print happens on the next line
print()

# convert combined data into a DataFrame
screener_df = pd.DataFrame(combined_data)

# join with the filtered_df - this adds exchange and exchangeName data
screener_df = screener_df.merge(filtered_df[['symbol', 'exchange', 'exchangeName']], on='symbol', how='left')

Total batches: 78
Progress: |██████████████████████████████████████████████████| 100.00% Complete


In [390]:
print(screener_df.head())
print()
print(f'Length: {len(screener_df.index)}')

  symbol     marketcap exchange                 exchangeName
0      A  3.930835e+10     XNYS  New York Stock Exchange Inc
1     AA  4.763415e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.584605e+07     XNAS           Nasdaq All Markets
3   AACI  8.848655e+07     XNAS           Nasdaq All Markets
4  AACIU  8.848655e+07     XNAS           Nasdaq All Markets

Length: 7790


In [412]:
# drop NAs
screener_df.dropna(inplace=True)

# determins the count of droped NAs
print(f'New length: {len(screener_df)}')
print()

screener_df.head(5)

New length: 7688



,symbol,marketcap,exchange,exchangeName
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc
2,AACG,4.210928e+07,XNAS,Nasdaq All Markets
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets


In [413]:
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

quote_data_list = []

def fetch_quote_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# initialize a counter for completed batches
completed_batches = 0

# define the length of the progress bar
progress_bar_length = 50

"""scalable endpoint variable data request"""
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_quote_data(batch_symbols, iex_key)
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        quote_data = batch_data.get(symbol, {}).get('quote', {})
        quote_data_list.append({
            'symbol': symbol, # KEY
            'latestPrice': quote_data.get('latestPrice', None),
            'close': quote_data.get('close', None), 
            'previousClose': quote_data.get('previousClose', None),
            'extendedPrice': quote_data.get('extendedPrice', None), 
            'extendedChange': quote_data.get('extendedChange', None), 
            'extendedChangePercent': quote_data.get('extendedChangePercent', None), 
            'latestVolume': quote_data.get('latestVolume', None), 
            'volume': quote_data.get('volume', None), 
            'previousVolume': quote_data.get('previousVolume', None), 
            'primaryExchange': quote_data.get('primaryExchange', None),
            'avgTotalVolume': quote_data.get('avgTotalVolume', None),
            'calculationPrice': quote_data.get('calculationPrice', None),
            'change': quote_data.get('change', None),
            'changePercent': quote_data.get('changePercent', None),
            'companyName': quote_data.get('companyName', None),
            # IEX real time prices - after hours without UTP authorization (maybe)
            'iexClose': quote_data.get('iexClose', None),
            'iexCloseTime': quote_data.get('iexCloseTime', None),
            'iexRealtimePrice': quote_data.get('iexRealtimePrice', None),
            'iexLastUpdated': quote_data.get('iexLastUpdated', None),
            'iexVolume': quote_data.get('iexVolume', None),
            # updates overkill
            'latestTime': quote_data.get('latestTime', None),
            'latestUpdate': quote_data.get('latestUpdate', None)})
    
    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
quote_df = pd.DataFrame(quote_data_list)

# MERGE DATA
screener_df = screener_df.merge(quote_df, on='symbol', how='left')

screener_df.head(5)

Total batches: 77


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,...,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,134.140,134.14,134.84,134.14,0.000,0.00000,...,-0.700,-0.00519,Agilent Technologies Inc.,134.14,1.708463e+12,134.14,1.708463e+12,43483.0,"February 20, 2024",1708462801959
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc,26.690,26.69,27.40,26.73,0.040,0.00150,...,-0.710,-0.02591,Alcoa Corp,26.70,1.708463e+12,26.70,1.708463e+12,153167.0,"February 20, 2024",1708462875282
2,AACG,4.210928e+07,XNAS,Nasdaq All Markets,1.341,1.46,1.46,1.36,0.019,0.01417,...,-0.119,-0.08151,ATA Creativity Global - ADR,1.46,1.708117e+12,0.00,0.000000e+00,0.0,3:46:36 PM,1708461996519
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,10.980,NaN,10.98,11.02,0.040,0.00364,...,0.000,0.00000,Armada Acquisition Corp I,10.93,1.707235e+12,0.00,0.000000e+00,0.0,"February 16, 2024",1708059600000
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.900,NaN,10.90,10.90,0.000,0.00000,...,0.000,0.00000,Armada Acquisition Corp I - Units (1 Ord & 1/2...,10.90,1.706906e+12,0.00,0.000000e+00,0.0,"February 2, 2024",1706850000000


In [414]:
def fetch_shares_outstanding_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=stats&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

# Assuming screener_df is a DataFrame containing the symbols
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

shares_outstanding_data_list = []

# initialize a counter for completed batches
completed_batches = 0
# define the length of the progress bar
progress_bar_length = 50

# Replace 'screener_df' with the actual DataFrame containing the symbols
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_shares_outstanding_data(batch_symbols, iex_key)
    
    # Process and append data for each symbol in the batch
    for symbol in batch_symbols:
        stats_data = batch_data.get(symbol, {}).get('stats', {})
        shares_outstanding_data_list.append({
            'symbol': symbol,
            'sharesOutstanding': stats_data.get('sharesOutstanding', None),
            'avg10Volume': stats_data.get('avg10Volume', None),
            'avg30Volume': stats_data.get('avg30Volume', None),
        })

    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
shares_outstanding_df = pd.DataFrame(shares_outstanding_data_list)

# MERGE DATA
screener_df = screener_df.merge(shares_outstanding_df, on='symbol', how='left')

screener_df.head(5)

Total batches: 77


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,...,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,sharesOutstanding,avg10Volume,avg30Volume
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,134.140,134.14,134.84,134.14,0.000,0.00000,...,134.14,1.708463e+12,134.14,1.708463e+12,43483.0,"February 20, 2024",1708462801959,293039707,1249900,1311345
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc,26.690,26.69,27.40,26.73,0.040,0.00150,...,26.70,1.708463e+12,26.70,1.708463e+12,153167.0,"February 20, 2024",1708462875282,178471908,5162225,5761875
2,AACG,4.210928e+07,XNAS,Nasdaq All Markets,1.341,1.46,1.46,1.36,0.019,0.01417,...,1.46,1.708117e+12,0.00,0.000000e+00,0.0,3:46:36 PM,1708461996519,31401405,42101,20623
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,10.980,NaN,10.98,11.02,0.040,0.00364,...,10.93,1.707235e+12,0.00,0.000000e+00,0.0,"February 16, 2024",1708059600000,8058884,461,14409
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.900,NaN,10.90,10.90,0.000,0.00000,...,10.90,1.706906e+12,0.00,0.000000e+00,0.0,"February 2, 2024",1706850000000,15000000,0,6983


In [394]:
"""ADD other data here before starting the market cap calculation"""

'ADD other data here before starting the market cap calculation'

In [415]:
# categorize market cap
def categorize_market_cap(x):
    if x < 300e6:  #  < 300 million
        return 'Micro-Cap'
    elif 300e6 <= x < 2e9:  # 300 million to 2 billion
        return 'Small-Cap'
    elif 2e9 <= x < 10e9:  # 2 billion to 10 billion
        return 'Mid-Cap'
    elif 10e9 <= x < 200e9:  # 10 billion to 200 billion
        return 'Large-Cap'
    elif x >= 200e9:  # >= 200 billion
        return 'Mega-Cap'
    else:
        return 'Unknown'  # handles negative or NaN values

# apply function to 'marketcap' column AND create new 'marketcapType' column
screener_df['marketcapType'] = screener_df['marketcap'].apply(categorize_market_cap)

screener_df.head(10)

,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,...,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,sharesOutstanding,avg10Volume,avg30Volume,marketcapType
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,134.140,134.14,134.840,134.1400,0.0000,0.00000,...,1.708463e+12,134.140,1.708463e+12,43483.0,"February 20, 2024",1708462801959,293039707,1249900,1311345,Large-Cap
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc,26.690,26.69,27.400,26.7300,0.0400,0.00150,...,1.708463e+12,26.700,1.708463e+12,153167.0,"February 20, 2024",1708462875282,178471908,5162225,5761875,Mid-Cap
2,AACG,4.210928e+07,XNAS,Nasdaq All Markets,1.341,1.46,1.460,1.3600,0.0190,0.01417,...,1.708117e+12,0.000,0.000000e+00,0.0,3:46:36 PM,1708461996519,31401405,42101,20623,Micro-Cap
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,10.980,NaN,10.980,11.0200,0.0400,0.00364,...,1.707235e+12,0.000,0.000000e+00,0.0,"February 16, 2024",1708059600000,8058884,461,14409,Micro-Cap
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.900,NaN,10.900,10.9000,0.0000,0.00000,...,1.706906e+12,0.000,0.000000e+00,0.0,"February 2, 2024",1706850000000,15000000,0,6983,Micro-Cap
5,AACT,5.270000e+08,XNYS,New York Stock Exchange Inc,10.540,10.51,10.520,10.5400,0.0000,0.00000,...,1.708108e+12,0.000,0.000000e+00,0.0,3:59:47 PM,1708462787273,50000000,140291,214072,Small-Cap
6,AACT=,5.260000e+08,XNYS,New York Stock Exchange Inc,10.629,NaN,10.629,10.6290,0.0000,0.00000,...,1.706714e+12,0.000,0.000000e+00,0.0,"February 15, 2024",1707973200000,50000000,600,4678,Small-Cap
7,AADI,4.684439e+07,XNAS,Nasdaq All Markets,1.910,1.91,1.900,1.9600,0.0500,0.02618,...,1.708463e+12,1.920,1.708463e+12,2735.0,"February 20, 2024",1708462800381,24525860,241834,227696,Micro-Cap
8,AADR,2.574630e+07,XNAS,Nasdaq All Markets,57.214,54.39,57.190,57.1300,-0.0840,-0.00147,...,1.707926e+12,0.000,0.000000e+00,0.0,3:59:48 PM,1708462788445,450000,1002,730,Micro-Cap
9,AAGR,4.629346e+07,XNAS,Nasdaq All Markets,0.800,0.80,0.829,0.8339,0.0339,0.04237,...,1.708463e+12,0.776,1.708463e+12,6681.0,"February 20, 2024",1708462800146,57866830,87929,107901,Micro-Cap


In [416]:
#print columns for screener_df
print(screener_df.columns)

Index(['symbol', 'marketcap', 'exchange', 'exchangeName', 'latestPrice',
       'close', 'previousClose', 'extendedPrice', 'extendedChange',
       'extendedChangePercent', 'latestVolume', 'volume', 'previousVolume',
       'primaryExchange', 'avgTotalVolume', 'calculationPrice', 'change',
       'changePercent', 'companyName', 'iexClose', 'iexCloseTime',
       'iexRealtimePrice', 'iexLastUpdated', 'iexVolume', 'latestTime',
       'latestUpdate', 'sharesOutstanding', 'avg10Volume', 'avg30Volume',
       'marketcapType'],
      dtype='object')


In [417]:
# create a columns filter
screener_cols = ['symbol', # KEY
                 # market cap
                'marketcap',
                'marketcapType', 
                # price(s)
                'close', 
                'latestPrice', 
                'extendedPrice',
                # relative change
                'calculationPrice',
                'change', #mandatory
                'changePercent', 
                # shares
                'sharesOutstanding', 
                # volume
                'latestVolume', 
                'avg10Volume',
                'avg30Volume',
                #info
                'exchange',
                'exchangeName',
                # IEX real time prices - after hours without UTP authorization
                'iexClose',
                'iexCloseTime',
                'iexRealtimePrice',
                #### ADD -  CHANGE % (CALCULATION - custom) ####
                'iexLastUpdated',
                'iexVolume',
                # IEX for updates checking overkill
                'latestTime',
                'latestUpdate']

screener_df_x = screener_df[screener_cols]

screener_df_x.head(25)

,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,...,avg30Volume,exchange,exchangeName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate
0,A,3.930835e+10,Large-Cap,134.14,134.140,134.1400,close,-0.700,-0.00519,293039707,...,1311345,XNYS,New York Stock Exchange Inc,134.140,1.708463e+12,134.140,1.708463e+12,43483.0,"February 20, 2024",1708462801959
1,AA,4.763415e+09,Mid-Cap,26.69,26.690,26.7300,close,-0.710,-0.02591,178471908,...,5761875,XNYS,New York Stock Exchange Inc,26.700,1.708463e+12,26.700,1.708463e+12,153167.0,"February 20, 2024",1708462875282
2,AACG,4.210928e+07,Micro-Cap,1.46,1.341,1.3600,sip,-0.119,-0.08151,31401405,...,20623,XNAS,Nasdaq All Markets,1.460,1.708117e+12,0.000,0.000000e+00,0.0,3:46:36 PM,1708461996519
3,AACI,8.848655e+07,Micro-Cap,NaN,10.980,11.0200,previousclose,0.000,0.00000,8058884,...,14409,XNAS,Nasdaq All Markets,10.930,1.707235e+12,0.000,0.000000e+00,0.0,"February 16, 2024",1708059600000
4,AACIU,8.848655e+07,Micro-Cap,NaN,10.900,10.9000,previousclose,0.000,0.00000,15000000,...,6983,XNAS,Nasdaq All Markets,10.900,1.706906e+12,0.000,0.000000e+00,0.0,"February 2, 2024",1706850000000
5,AACT,5.270000e+08,Small-Cap,10.51,10.540,10.5400,sip,0.020,0.00190,50000000,...,214072,XNYS,New York Stock Exchange Inc,10.525,1.708108e+12,0.000,0.000000e+00,0.0,3:59:47 PM,1708462787273
6,AACT=,5.260000e+08,Small-Cap,NaN,10.629,10.6290,previousclose,0.000,0.00000,50000000,...,4678,XNYS,New York Stock Exchange Inc,10.570,1.706714e+12,0.000,0.000000e+00,0.0,"February 15, 2024",1707973200000
7,AADI,4.684439e+07,Micro-Cap,1.91,1.910,1.9600,close,0.010,0.00526,24525860,...,227696,XNAS,Nasdaq All Markets,1.920,1.708463e+12,1.920,1.708463e+12,2735.0,"February 20, 2024",1708462800381
8,AADR,2.574630e+07,Micro-Cap,54.39,57.214,57.1300,sip,0.024,0.00042,450000,...,730,XNAS,Nasdaq All Markets,56.570,1.707926e+12,0.000,0.000000e+00,0.0,3:59:48 PM,1708462788445
9,AAGR,4.629346e+07,Micro-Cap,0.80,0.800,0.8339,close,-0.029,-0.03498,57866830,...,107901,XNAS,Nasdaq All Markets,0.776,1.708463e+12,0.776,1.708463e+12,6681.0,"February 20, 2024",1708462800146


In [418]:
# create five (5) dataframes for each marketcap type
microcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Micro-Cap']
smallcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Small-Cap']
midcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Mid-Cap']
largecap_df = screener_df_x[screener_df_x['marketcapType'] == 'Large-Cap']
megacap_df = screener_df_x[screener_df_x['marketcapType'] == 'Mega-Cap']

In [420]:
largecap_df.head(5)

,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,...,avg30Volume,exchange,exchangeName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate
0,A,3.930835e+10,Large-Cap,134.14,134.14,134.14,close,-0.70,-0.00519,293039707,...,1311345,XNYS,New York Stock Exchange Inc,134.14,1.708463e+12,134.14,1.708463e+12,43483.0,"February 20, 2024",1708462801959
29,ABEV,4.032981e+10,Large-Cap,2.56,2.56,2.58,close,0.01,0.00392,15753833284,...,8719356,XNYS,New York Stock Exchange Inc,2.57,1.708463e+12,2.57,1.708463e+12,532248.0,"February 20, 2024",1708462801887
38,ABNB,9.516818e+10,Large-Cap,148.15,148.15,146.50,close,-4.36,-0.02859,642377183,...,6142626,XNAS,Nasdaq All Markets,148.15,1.708463e+12,148.15,1.708463e+12,167001.0,"February 20, 2024",1708462800460
70,ACGL,3.185769e+10,Large-Cap,85.37,85.37,85.13,close,-0.74,-0.00859,373171909,...,1645827,XNAS,Nasdaq All Markets,85.42,1.708463e+12,85.42,1.708463e+12,88379.0,"February 20, 2024",1708462800000
71,ACGLN,3.229315e+10,Large-Cap,20.27,20.27,20.18,close,0.10,0.00496,20000000,...,28120,XNAS,Nasdaq All Markets,20.17,1.708462e+12,20.17,1.708462e+12,566.0,"February 20, 2024",1708464600000


In [434]:
"""
Function that queries dataframe based on variable ranges (min-max)
"""
def filter_stocks(df, 
                  # PRICE
                  price_min=None, 
                  price_max=None, 
                  # CHANGE 
                  change_min_percent=None,
                  change_max_percent=None,
                  # VOLUME
                  volume_min=None, 
                  # SHARES OUTSTANDING (approx. Float)
                  shares_outstanding_min=None, 
                  shares_outstanding_max=None):
    # PRICE
    if price_min is not None:
        df = df[df['latestPrice'] >= price_min]
    if price_max is not None:
        df = df[df['latestPrice'] <= price_max]
    # CHANGE
    if change_min_percent is not None:
        df = df[df['changePercent'] >= change_min_percent]
    if change_max_percent is not None:
        df = df[df['changePercent'] <= change_max_percent]
    # VOLUME
    if volume_min is not None:
        df = df[df['latestVolume'] >= volume_min]
    # SHARES OUTSTANDING (approx. Float)
    if shares_outstanding_min is not None:
        df = df[df['sharesOutstanding'] >= shares_outstanding_min]
    if shares_outstanding_max is not None:
        df = df[df['sharesOutstanding'] <= shares_outstanding_max]
    
    return df

In [447]:
"""
Large Cap Stocks
"""
# test this out with an appropriate query for large-cap stocks
largecap_df = filter_stocks(largecap_df, 
                                     price_min=25, 
                                     price_max=200, 
                                     change_min_percent=.01, 
                                     change_max_percent=None, 
                                     volume_min=1000000, 
                                     shares_outstanding_min=100000000, 
                                     shares_outstanding_max=500000000)

# sort in descending order by changePercent
largecap_df = largecap_df.sort_values('changePercent', ascending=False)

largecap_df.head(20)

,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,...,avg30Volume,exchange,exchangeName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate
1994,DFS,3.111221e+10,Large-Cap,124.42,124.42,124.05,close,13.93,0.12607,250057955,...,1851805,XNYS,New York Stock Exchange Inc,124.445,1.708463e+12,124.445,1.708463e+12,991862.0,"February 20, 2024",1708462802321
2050,DLTR,3.187249e+10,Large-Cap,146.29,146.29,146.16,close,2.92,0.02037,217871937,...,2078908,XNAS,Nasdaq All Markets,146.280,1.708463e+12,146.280,1.708463e+12,77526.0,"February 20, 2024",1708462800210
332,ALNY,1.885283e+10,Large-Cap,149.69,149.69,147.50,close,2.69,0.01830,125945793,...,1060832,XNAS,Nasdaq All Markets,149.800,1.708463e+12,149.800,1.708463e+12,100534.0,"February 20, 2024",1708462800353
2099,DRI,1.973980e+10,Large-Cap,165.31,165.31,165.31,close,2.85,0.01754,119410813,...,909768,XNYS,New York Stock Exchange Inc,165.340,1.708463e+12,165.340,1.708463e+12,96599.0,"February 20, 2024",1708462802201
5398,PHM,2.185607e+10,Large-Cap,103.04,103.04,104.00,close,1.59,0.01567,212112455,...,2029367,XNYS,New York Stock Exchange Inc,103.080,1.708463e+12,103.080,1.708463e+12,62201.0,"February 20, 2024",1708462844682
2018,DHI,4.760583e+10,Large-Cap,143.47,143.47,144.05,close,2.20,0.01557,331817282,...,3192064,XNYS,New York Stock Exchange Inc,143.515,1.708463e+12,143.515,1.708463e+12,80699.0,"February 20, 2024",1708462802120
6277,SJM,1.336138e+10,Large-Cap,125.88,125.88,125.88,close,1.79,0.01443,106143808,...,1189351,XNYS,New York Stock Exchange Inc,125.910,1.708463e+12,125.910,1.708463e+12,31127.0,"February 20, 2024",1708462801928
6857,TOL,1.079911e+10,Large-Cap,103.55,103.55,105.72,close,1.37,0.01341,104288855,...,1323612,XNYS,New York Stock Exchange Inc,103.580,1.708463e+12,103.580,1.708463e+12,161180.0,"February 20, 2024",1708463021778
3412,HSY,3.969734e+10,Large-Cap,193.57,193.57,194.00,close,2.41,0.01261,205080000,...,1949257,XNYS,New York Stock Exchange Inc,193.500,1.708463e+12,193.500,1.708463e+12,58149.0,"February 20, 2024",1708462802165


END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END


In [407]:
"""
IMPORTANT: Need UTP autorization to continue
https://iexcloud.io/documentation/using-core-data/getting-nasdaq-listed-utp-otc-stock-data.html
Step 1: https://www.utpplan.com/DOC/VendorAgreement.pdf
Step 2: https://www.utpplan.com/datafeed_approval
Step 3: https://www.utpplan.com/system_application

"""

# create a dataframe from utp_auth_columns.xlsx
utp_auth_df = pd.read_excel('utp_auth_columns.xlsx')
# replace NA with " " 
utp_auth_df = utp_auth_df.fillna(" ")

utp_auth_df

,utp_auth_required,humbled_trader_TARGET,iex_real-time_alternative,TRANSFORMED_current
0,close,Symbol,,
1,closeTime,Price ($),,
2,delayedPrice,Float (shares),,
3,delayedPriceTime,Change Close (%),,
4,extendedPrice,Volume Today (shares),,
5,extendedPriceTime,Avereage Volume (5day) (shares/day),,
6,extendedChange,Market Cap ($),,
7,extendedChangePercent,Held By Institutions (%),,
8,high,Sector,,
9,low,Company Name,,


In [399]:
# # DEV - marketcap groupby count
# marketcap_counts = screener_df['marketcapType'].value_counts()

# # convert SERIES to DF
# marketcap_counts_df = marketcap_counts.reset_index()
# marketcap_counts_df.columns = ['marketcapType', 'count']

# # add percentage column
# total_count = marketcap_counts.sum()
# marketcap_counts_df['percentage'] = (marketcap_counts_df['count'] / total_count) * 100

# # print
# marketcap_counts_df

In [400]:
# # DEV - checker
# total_count_CHECK = marketcap_counts_df['count'].sum()

# # if statement comparing the two values, return 'Data is correct' if they match, otherwise return 'Data is incorrect'
# if total_count_CHECK == len(screener_df):
#     print('Data lengths match! SUCCESS')
# else:    
#     print('Data mismatch! ERROR')
#     print()
#     print(f'Code checker: {total_count_CHECK}')
#     print(f'Data length: {len(screener_df)}')

In [401]:
# def print_available_stats_parameters(symbol, iex_key):
#     url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={iex_key}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()
#         print(json.dumps(data, indent=4, sort_keys=True))
#     else:
#         print(f"Failed to fetch stats for {symbol}. Status code: {response.status_code}")

# pltr = 'PLTR'
# print_available_stats_parameters(pltr, iex_key)

In [402]:
# def fetch_deep_trades_data(symbols_batch, iex_key):
#     symbols_str = ','.join(symbols_batch)
#     url = f'https://cloud.iexapis.com/stable/deep/trades?symbols={symbols_str}&token={iex_key}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return {}

# # Assuming screener_df is a DataFrame containing the symbols
# batch_size = 100
# # Example for illustrative purposes
# # screener_df = pd.DataFrame({'symbol': ['AAPL', 'MSFT', 'GOOGL']})
# total_batches = math.ceil(len(screener_df) / batch_size)
# print(f"Total batches: {total_batches}")

# deep_data_list = []

# for i in range(0, len(screener_df), batch_size):
#     batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
#     batch_data = fetch_deep_trades_data(batch_symbols, iex_key)
    
#     # Process and infer volume data for each symbol in the batch
#     for symbol in batch_symbols:
#         if symbol in batch_data and batch_data[symbol]:
#             volume = sum(trade['size'] for trade in batch_data[symbol])
#             deep_data_list.append({
#                 'symbol': symbol,
#                 'volume': volume,
#             })

# # Convert the combined data into a DataFrame
# iex_deep_df = pd.DataFrame(deep_data_list)

# print(iex_deep_df.head(20)) # data check
# print(len(iex_deep_df.index)) # data check

In [403]:
# print(f"screener_df column count: {len(screener_df.columns)}") # col count
# screener_df = screener_df.merge(shares_outstanding_df, on='symbol', how='left')

In [404]:
# # My Watchlist
# watchlist = ['PLTR', 'TSLA', 'NOW', 'SNOW','FB', 'NVDA', 'PYPL', 'ADBE', 'NFLX']
# watchlist_df = screener_df[screener_df['symbol'].isin(watchlist)]

# watchlist_df

In [405]:
# # DEV - API endpoing parameter availability. 
# schema_base_test_url = 'https://cloud.iexapis.com/stable'
# test_symbol = 'PLTR' # test ticker symbol
# schema_test_url = f"{schema_base_test_url}/stock/{test_symbol}/quote?schema=true&token={iex_key}"
# # GET request
# schema_test_response = requests.get(schema_test_url)

# print(json.dumps(schema_test_response.json(), indent=4))


IDEAS
1. Robust value score (RBV)
2. create data frames for each market cap
3. create a function that does math and querying based on rules / conditions
4. apply function to the market cap data sets
5. Look at other STATS now that the API is connected for the sharesOutstanding

NOTES
1. Float
1. Free Flt Mkt Cap
1. Free Flt
1. Shares outstanding

MANDATORY FIELDS
1. 